In [342]:
import pandas as pd
import numpy as np

marketValue_df = pd.read_csv("https://storm.cis.fordham.edu/~jcao42/2010-2025.csv")

In [344]:
marketValue_df.head()

,date,club_name,market_value,current_value
0,2010-11-01,Manchester City,€360.30m,€1.31bn
1,2010-11-01,Arsenal FC,€303.00m,€1.13bn
2,2010-11-01,Liverpool FC,€275.00m,€993.50m
3,2010-11-01,Chelsea FC,€360.00m,€922.00m
4,2010-11-01,Tottenham Hotspur,€282.70m,€836.10m


#processing data for marketValue data sets


In [347]:
marketValue_df.info() #overlook of 2010-2025 market value datasets

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6900 entries, 0 to 6899
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   date           6900 non-null   object
 1   club_name      6900 non-null   object
 2   market_value   6900 non-null   object
 3   current_value  6900 non-null   object
dtypes: object(4)
memory usage: 215.8+ KB


In [349]:
parsed_df = marketValue_df.copy()

In [351]:
parsed_df["date"] = pd.to_datetime(marketValue_df["date"])
#define a function to translate money symbol to numerical value
def parse_value(value_str):
    s = value_str.strip().replace("€","")
    if s == "-":
        return np.nan
    if s.endswith("bn"):
        numeric_part = s.replace("bn", "")
        return float(numeric_part)*1e9
    elif s.endswith("m"):
        numeric_part = s.replace("m","")
        return float(numeric_part) * 1e6
    else:
        return float(s)
parsed_df["market_value"] = marketValue_df["market_value"].apply(parse_value)
parsed_df["current_value"] = marketValue_df["current_value"].apply(parse_value)

In [353]:
marketValue_df.head()

,date,club_name,market_value,current_value
0,2010-11-01,Manchester City,€360.30m,€1.31bn
1,2010-11-01,Arsenal FC,€303.00m,€1.13bn
2,2010-11-01,Liverpool FC,€275.00m,€993.50m
3,2010-11-01,Chelsea FC,€360.00m,€922.00m
4,2010-11-01,Tottenham Hotspur,€282.70m,€836.10m


In [355]:
parsed_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6900 entries, 0 to 6899
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   date           6900 non-null   datetime64[ns]
 1   club_name      6900 non-null   object        
 2   market_value   6680 non-null   float64       
 3   current_value  6900 non-null   float64       
dtypes: datetime64[ns](1), float64(2), object(1)
memory usage: 215.8+ KB


In [357]:
parsed_df.isna().sum() #check missing value

date               0
club_name          0
market_value     220
current_value      0
dtype: int64

In [359]:
#since the data update every 15 days, we consider it is relatively short. So, use data from 15 days ago and 1 month ago will be a greate way to replace missing value.
parsed_df = parsed_df.sort_values(["club_name", "date"]).reset_index(drop=True)
parsed_df["market_value"] = parsed_df.groupby("club_name")["market_value"].ffill()

In [361]:
parsed_df.isna().sum() #check missing value, now it should be 0

date             0
club_name        0
market_value     0
current_value    0
dtype: int64

In [363]:
RawData_df = pd.read_csv("https://storm.cis.fordham.edu/~jcao42/RawDataPremier.csv")

In [365]:
RawData_df.tail()

,Team_ID,TeamName,Season,Wins,Losses,Goals,Goal_Conceded,GoalsPerMatch,Shots,ShotsOnTarget,...,TackleSuccess%,BlockedShots,Interceptions,Clearances,HeadedClearance,AerialBattle/DuelsWon,ErrorsLeadingToGoal,OwnGoals,Champion,FiredManager
193,15,Nottingham-Forest,2023/24,9,20,49,67,1.29,451,148,...,59%,116,331,910,440,"2,554",8,2,0,1
194,18,Sheffield-United,2023/24,3,28,35,104,0.92,365,129,...,57%,105,347,997,451,"2,466",9,7,0,1
195,21,Tottenham-Hotspur,2023/24,20,12,74,61,1.95,584,215,...,62%,171,355,652,334,"2,319",6,3,0,0
196,25,West-Ham-United,2023/24,14,14,60,74,1.58,450,154,...,62%,112,361,862,417,"2,585",3,4,0,0
197,38,Wolverhampton-Wanderers,2023/24,13,18,50,65,1.32,429,160,...,58%,111,284,786,406,"2,450",3,1,0,0


In [367]:
manager_Changes_df = pd.read_csv("https://storm.cis.fordham.edu/~jcao42/manager_changes_2010_2025.csv")

In [369]:
manager_Changes_df.head()

,season,change_type,club,coach_name,role,matchday,rank1,ppg1,last_match_home,last_match_away,last_match_score,leaving_date,days_in_charge,successor_name,successor_role,rank2,ppg2,first_match_home,first_match_away,first_match_score
0,2010,Change(s) in coach at the end of the season,Chelsea FC,Carlo Ancelotti,Manager,38,2,1.87,Everton FC,Chelsea FC,1:0,"Jun 30, 2011",729,André Villas-Boas,Manager,-,-,NaN,NaN,NaN
1,2010,Change(s) in coach at the end of the season,Birmingham City,Alex McLeish,Manager,38,18,1.03,Tottenham Hotspur,Birmingham City,2:1,"Jun 12, 2011",1293,Chris Hughton,Manager,-,-,NaN,NaN,NaN
2,2010,Change(s) in coach at the end of the season,Fulham FC,Mark Hughes,Manager,38,8,1.29,Fulham FC,Arsenal FC,2:2,"Jun 2, 2011",308,Martin Jol,Manager,-,-,NaN,NaN,NaN
3,2010,Change(s) in coach at the end of the season,Aston Villa,Gérard Houllier,Manager,38,9,1.21,Aston Villa,Liverpool FC,1:0,"Jun 1, 2011",265,Alex McLeish,Manager,-,-,NaN,NaN,NaN
4,2010,Change(s) in coach at the end of the season,West Ham United,Kevin Keen,Caretaker Manager,38,20,0.00,West Ham United,Sunderland AFC,0:3,"May 31, 2011",16,Sam Allardyce,Manager,-,-,NaN,NaN,NaN


In [371]:
manager_Changes_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 248 entries, 0 to 247
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   season             248 non-null    int64 
 1   change_type        248 non-null    object
 2   club               248 non-null    object
 3   coach_name         248 non-null    object
 4   role               248 non-null    object
 5   matchday           248 non-null    object
 6   rank1              248 non-null    object
 7   ppg1               248 non-null    object
 8   last_match_home    231 non-null    object
 9   last_match_away    231 non-null    object
 10  last_match_score   231 non-null    object
 11  leaving_date       248 non-null    object
 12  days_in_charge     248 non-null    int64 
 13  successor_name     248 non-null    object
 14  successor_role     248 non-null    object
 15  rank2              248 non-null    object
 16  ppg2               248 non-null    object
 1

In [373]:
#in the role, caretaker manager will not be considered since it not benefits for our analysis.
manager_Changes_df = manager_Changes_df[~manager_Changes_df["role"].str.lower().str.contains("caretaker")]
#change data as datetime
manager_Changes_df["leaving_date"] = pd.to_datetime(manager_Changes_df["leaving_date"])

In [375]:
manager_Changes_df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 171 entries, 0 to 247
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   season             171 non-null    int64         
 1   change_type        171 non-null    object        
 2   club               171 non-null    object        
 3   coach_name         171 non-null    object        
 4   role               171 non-null    object        
 5   matchday           171 non-null    object        
 6   rank1              171 non-null    object        
 7   ppg1               171 non-null    object        
 8   last_match_home    164 non-null    object        
 9   last_match_away    164 non-null    object        
 10  last_match_score   164 non-null    object        
 11  leaving_date       171 non-null    datetime64[ns]
 12  days_in_charge     171 non-null    int64         
 13  successor_name     171 non-null    object        
 14  successor_role 

In [377]:
#check if they been fired
def determine_fired(change_type):
    if pd.isna(change_type):
        return np.nan
    change_type = change_type.lower()
    if "during the season" in change_type:
        return 1
    elif "at the end of the season" in change_type:
        return 0
    else:
        return np.nan  

manager_Changes_df["fired"] = manager_Changes_df["change_type"].apply(determine_fired)

In [379]:
#df after cleaning 
manager_clean = manager_Changes_df[[
    "season", "club", "coach_name", "leaving_date", "days_in_charge", "fired"
]].rename(columns={
    "club": "club_name",
    "coach_name": "manager_name"
})


In [381]:
manager_clean.head()

,season,club_name,manager_name,leaving_date,days_in_charge,fired
0,2010,Chelsea FC,Carlo Ancelotti,2011-06-30,729,0.0
1,2010,Birmingham City,Alex McLeish,2011-06-12,1293,0.0
2,2010,Fulham FC,Mark Hughes,2011-06-02,308,0.0
3,2010,Aston Villa,Gérard Houllier,2011-06-01,265,0.0
5,2010,West Ham United,Avram Grant,2011-05-15,318,1.0


In [383]:
manager_clean["fired"].value_counts(dropna=False)

fired
1.0    111
0.0     54
NaN      6
Name: count, dtype: int64

In [385]:
#deal with NaN value
#check record
print(manager_clean[manager_clean["fired"].isna()])

# what if we delete it
manager_clean = manager_clean.dropna(subset=["fired"])
print(manager_clean["fired"].value_counts(dropna=False))


     season                club_name     manager_name leaving_date  \
15     2010              Aston Villa   Martin O'Neill   2010-08-09   
76     2014           Crystal Palace       Tony Pulis   2014-08-14   
108    2016                Hull City      Steve Bruce   2016-07-22   
109    2016           Sunderland AFC    Sam Allardyce   2016-07-21   
144    2018               Chelsea FC    Antonio Conte   2018-07-13   
227    2023  Wolverhampton Wanderers  Julen Lopetegui   2023-08-08   

     days_in_charge  fired  
15             1466    NaN  
76              263    NaN  
108            1482    NaN  
109             286    NaN  
144             742    NaN  
227             267    NaN  
fired
1.0    111
0.0     54
Name: count, dtype: int64


In [387]:
#we can put some values by our self base on real situation. 
fired_manual = {
    "Martin O'Neill": 0,
    "Tony Pulis": 1,
    "Steve Bruce": 0,
    "Sam Allardyce": 0,
    "Antonio Conte": 1,
    "Julen Lopetegui": 0
}

manager_clean["fired"] = manager_clean.apply(
    lambda row: fired_manual[row["manager_name"]] if pd.isna(row["fired"]) and row["manager_name"] in fired_manual else row["fired"],
    axis=1
)

In [389]:
print(manager_clean["fired"].value_counts())

fired
1.0    111
0.0     54
Name: count, dtype: int64


In [391]:
print(manager_clean[["club_name", "season"]].drop_duplicates().head())
print(parsed_df[["club_name", "date"]].drop_duplicates().head())

         club_name  season
0       Chelsea FC    2010
1  Birmingham City    2010
2        Fulham FC    2010
3      Aston Villa    2010
5  West Ham United    2010
         club_name       date
0  AFC Bournemouth 2010-11-01
1  AFC Bournemouth 2010-11-15
2  AFC Bournemouth 2010-12-01
3  AFC Bournemouth 2010-12-15
4  AFC Bournemouth 2011-01-01


In [393]:
Income_Expanditures_df= pd.read_csv("https://storm.cis.fordham.edu/~jcao42/PremierLeague_Transfers_Income_expenditures_2014_2024.csv")

In [395]:
Income_Expanditures_df.head()

,year,club,expenditure,income,balance
0,2014,Manchester United,€195.35m,€42.90m,€-152.45m
1,2014,Liverpool FC,€151.43m,€91.97m,€-59.46m
2,2014,Chelsea FC,€106.70m,€96.51m,€-10.19m
3,2014,Arsenal FC,€101.73m,€25.20m,€-76.53m
4,2014,Southampton FC,€82.20m,€119.83m,€37.63m


In [397]:
Income_Expanditures_df.tail()

,year,club,expenditure,income,balance
215,2024,Wolverhampton Wanderers,€74.40m,€112.50m,€38.10m
216,2024,Newcastle United,€68.20m,€76.20m,€8.00m
217,2024,Everton FC,€50.20m,€83.65m,€33.45m
218,2024,Liverpool FC,€42.00m,€47.00m,€5.00m
219,2024,Manchester City,€25.00m,€141.00m,€116.00m


In [399]:
Income_Expanditures_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220 entries, 0 to 219
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   year         220 non-null    int64 
 1   club         220 non-null    object
 2   expenditure  220 non-null    object
 3   income       220 non-null    object
 4   balance      220 non-null    object
dtypes: int64(1), object(4)
memory usage: 8.7+ KB


In [401]:
marketValue_df[marketValue_df['club_name'] == 'Manchester City']

,date,club_name,market_value,current_value
0,2010-11-01,Manchester City,€360.30m,€1.31bn
20,2010-11-15,Manchester City,€357.30m,€1.31bn
40,2010-12-01,Manchester City,€357.30m,€1.31bn
60,2010-12-15,Manchester City,€357.30m,€1.31bn
80,2011-01-01,Manchester City,€357.30m,€1.31bn
...,...,...,...,...
6800,2025-01-01,Manchester City,€1.21bn,€1.31bn
6820,2025-01-15,Manchester City,€1.21bn,€1.31bn
6840,2025-02-01,Manchester City,€1.29bn,€1.31bn
6860,2025-02-15,Manchester City,€1.30bn,€1.31bn


In [403]:
RawData_df['Season'] = RawData_df['Season'].str.slice(0, 4)
RawData_df['TeamName'] = RawData_df['TeamName'].str.replace('-',' ')

In [405]:
RawData_df = RawData_df.rename(columns={
    "TeamName": "club_name",
    "Season": "season"
})

In [407]:
RawData_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 198 entries, 0 to 197
Data columns (total 32 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Team_ID                198 non-null    int64  
 1   club_name              198 non-null    object 
 2   season                 198 non-null    object 
 3   Wins                   198 non-null    int64  
 4   Losses                 198 non-null    int64  
 5   Goals                  198 non-null    int64  
 6   Goal_Conceded          198 non-null    int64  
 7   GoalsPerMatch          198 non-null    float64
 8   Shots                  198 non-null    int64  
 9   ShotsOnTarget          198 non-null    int64  
 10  ShootingAccuracy%      198 non-null    object 
 11  PenaltiesScored        198 non-null    int64  
 12  BigChancesCreated      198 non-null    int64  
 13  HitWoodwork            198 non-null    int64  
 14  YellowCards            198 non-null    int64  
 15  RedCar

In [409]:
RawData_df['season'] = RawData_df['season'].astype('int64')

In [411]:
manager_clean.head()

,season,club_name,manager_name,leaving_date,days_in_charge,fired
0,2010,Chelsea FC,Carlo Ancelotti,2011-06-30,729,0.0
1,2010,Birmingham City,Alex McLeish,2011-06-12,1293,0.0
2,2010,Fulham FC,Mark Hughes,2011-06-02,308,0.0
3,2010,Aston Villa,Gérard Houllier,2011-06-01,265,0.0
5,2010,West Ham United,Avram Grant,2011-05-15,318,1.0


In [413]:
RawData_df.head()

,Team_ID,club_name,season,Wins,Losses,Goals,Goal_Conceded,GoalsPerMatch,Shots,ShotsOnTarget,...,TackleSuccess%,BlockedShots,Interceptions,Clearances,HeadedClearance,AerialBattle/DuelsWon,ErrorsLeadingToGoal,OwnGoals,Champion,FiredManager
0,1,Arsenal,2014,22,7,71,36,1.87,610,227,...,75%,171,762,"1,006",565,"3,242",8,1,0,0
1,4,Chelsea,2014,26,3,73,32,1.92,564,210,...,81%,148,376,"1,027",611,"3,075",3,1,1,0
2,2,Aston Villa,2014,10,20,31,57,0.82,418,125,...,75%,104,487,"1,130",661,"2,726",10,1,0,1
3,43,Burnley,2014,7,19,28,53,0.74,430,125,...,73%,123,568,"1,458",999,"3,105",1,2,0,0
4,6,Crystal Palace,2014,13,16,47,51,1.24,441,139,...,78%,132,600,"1,270",779,"3,350",12,0,0,1


In [417]:
df_merge = pd.merge(RawData_df, manager_clean, on=['club_name', 'season'], how='left')

In [419]:
df_merge.head()

,Team_ID,club_name,season,Wins,Losses,Goals,Goal_Conceded,GoalsPerMatch,Shots,ShotsOnTarget,...,HeadedClearance,AerialBattle/DuelsWon,ErrorsLeadingToGoal,OwnGoals,Champion,FiredManager,manager_name,leaving_date,days_in_charge,fired
0,1,Arsenal,2014,22,7,71,36,1.87,610,227,...,565,"3,242",8,1,0,0,NaN,NaT,NaN,NaN
1,4,Chelsea,2014,26,3,73,32,1.92,564,210,...,611,"3,075",3,1,1,0,NaN,NaT,NaN,NaN
2,2,Aston Villa,2014,10,20,31,57,0.82,418,125,...,661,"2,726",10,1,0,1,Paul Lambert,2015-02-11,984.0,1.0
3,43,Burnley,2014,7,19,28,53,0.74,430,125,...,999,"3,105",1,2,0,0,NaN,NaT,NaN,NaN
4,6,Crystal Palace,2014,13,16,47,51,1.24,441,139,...,779,"3,350",12,0,0,1,Neil Warnock,2014-12-27,122.0,1.0
